# Leveraging Agents with Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. You can also run on a local setup, as long as you have the right IAM credentials to invoke the Claude model via Bedrock*

---

In this demo notebook, we demonstrate an implementation of Function Calling with Anthropic's Claude models via Bedrock. This notebook is inspired by the [original work](https://drive.google.com/drive/folders/1-94Fa3HxEMkxkwKppe8lp_9-IXXvsvv1) by the Anthropic Team and modified it for use with Amazon Bedrock.

This notebook need access to anthropic.claude-3-sonnet-20240229-v1:0 model in Bedrock

---

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. These use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries and can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. usuallythey are augmented by fetching information from various channels such as websites, social media platforms, and messaging apps which involve a complex workflow as shown below


### Chatbot using Amazon Bedrock

![Amazon Bedrock - Agents Interface](./images/agents.jpg)




### Building  - Key Elements

The first process in a building a contextual-aware chatbot is to identify the tools which can be called by the LLM's. 

Second process is the user request orchestration , interaction,  invoking and returing the results

### Architecture [Weather lookup]
We Search and look for the Latitude and Longitude and then invoke the weather app to get predictions

![Amazon Bedrock - Agents Interface](./images/weather.jpg)

#### Please un comment and install the libraries below if you do not have these 

In [ ]:
!pip install langchain==0.1.17
!pip install langchain-anthropic
!pip install boto3==1.34.95
!pip install faiss-cpu==1.8.0

#### To install the langchain-aws

you can run the `pip install langchain-aws`

to get the latest release use these commands below

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you have the required libraries and access to internet for the weather api's in this notebook. ⚠️ ⚠️ ⚠️


In [3]:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(assumed_role: Optional[str] = None, region: Optional[str] = 'us-east-1',runtime: Optional[bool] = True,external_id=None, ep_url=None):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides 
    """
    target_region = region

    print(f"Create new client\n  Using region: {target_region}:external_id={external_id}: ")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        if external_id:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
                ExternalId=external_id
            )
        else:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
            )
        print(f"Using role: {assumed_role} ... sts::successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    if ep_url:
        bedrock_client = session.client(service_name=service_name,config=retry_config,endpoint_url = ep_url, **client_kwargs )
    else:
        bedrock_client = session.client(service_name=service_name,config=retry_config, **client_kwargs )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [ ]:
import json
import os
import sys

import boto3
import botocore



# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


bedrock_runtime = get_bedrock_client() #
#     assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
#     region=os.environ.get("AWS_DEFAULT_REGION", None)
# )

### Anthropic Claude

#### Input

```json

"messages": [
    {"role": "user", "content": "Hello, Claude"},
    {"role": "assistant", "content": "Hello!"},
    {"role": "user", "content": "Can you describe LLMs to me?"}
        
]
{
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 100,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 0.9
} 
```

#### Output

```json
{
    'id': 'msg_01T',
    'type': 'message',
    'role': 'assistant',
    'content': [
        {
            'type': 'text',
            'text': 'Sure, the concept...'
        }
    ],
    'model': 'model_id',
    'stop_reason': 'max_tokens',
    'stop_sequence': None,
    'usage': {'input_tokens':xy, 'output_tokens': yz}}
```






### Bedrock model

Anthropic Claude

The key for this to work is to let LLM which is Claude models know about a set of `tools` that it has available i.e. functions it can call between a set of tags. This is possible because Anthropic's Claude models have been extensively trained on such tags in its training corpus.

Then present a way to call the tools in a step by step fashion till it gets the right answer. We create a set of callable functions , below e present a sample functions which can be modified to suit your needs



#### Helper function to pretty print

In [5]:
from io import StringIO
import sys
import textwrap
from langchain.llms.bedrock import Bedrock
from typing import Optional, List, Any
from langchain.callbacks.manager import CallbackManagerForLLMRun

def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))





## Section 1. Connectivity and invocation

**Invoke the model to ensure connectivity** 

In [ ]:
import json 
modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"

messages=[
    { 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': "What is quantum mechanics? "
        }]
    },
    { 
        "role":'assistant', 
        "content":[{
            'type':'text',
            'text': "It is a branch of physics that describes how matter and energy interact with discrete energy values "
        }]
    },
    { 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': "Can you explain a bit more about discrete energies?"
        }]
    }
]
body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 0.9
        }  
    )  
    
response = bedrock_runtime.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
print_ww(response_body)

In [ ]:

def test_sample_claude_invoke(prompt_str,boto3_bedrock ):
    modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
    messages=[{ 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': prompt_str
        }]
    }]
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 0.9
        }  
    )  
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId)
    response_body = json.loads(response.get('body').read())
    return response_body


test_sample_claude_invoke("what is quantum mechanics", bedrock_runtime)   

### Invoke Langchain with `Chain` using BedrockChat class

In [ ]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm_chat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", 
    model_kwargs={"temperature": 0.1},
    #region_name="us-west-2",
    client=bedrock_runtime,
)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm_chat, verbose=True, memory=memory
)

resp = conversation.predict(input="Hi there!")
print_ww(resp)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import ChatOpenAI

modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
cl_llm = BedrockChat(
    model_id=modelId,
    client=bedrock_runtime,
    #model_kwargs={"max_tokens_to_sample": 100},
    model_kwargs={"temperature": 0.1},
)
memory = ConversationBufferMemory()
conversation = ConversationChain( llm=cl_llm, verbose=True, memory=memory)


prompt_t = ChatPromptTemplate.from_messages([("human", "Explain this  {question}.")])
chain_t = prompt_t | cl_llm | StrOutputParser()

resp = chain_t.invoke({'question':"what is quantum physics"}) # has to match the variables
print_ww(resp)

## Section 2 -- Tooling

###  First we show how tools work and how is it done natively

Create a set of helper function

we will create a set of functions which we can the re use in our application
1. We will need to create a prompt template. This template helps Bedrock models understand the tools and how to invoke them.
2. Create a method to read the available tools and add it to the prompt being used to invoke Claude
3. Call function which will be responsbile to actually invoke the function with the `right` parameters
4. Format Results for helping the Model leverage the results for summarization
5. `Add to prompt`. The result which come back need to be added to the the prompt and model invoked again to get the right results

[See this notebook for more details](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/rag-solutions/rag-foundations-workshop/notebooks/05_agent_based_text_generation.ipynb)

In [7]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

### Add Tools

Recursively add the available tools

### With LangChain

Now use langchain and annotations to create the tools and invoke the functions. we will call weather api. The first tool will be used to look up the Latitude and second to get the weather

In [8]:
import requests

from langchain.tools import tool
from langchain.tools import StructuredTool
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

@tool ("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params,headers=headers).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None
    
@tool ("get_weather")
def get_weather(latitude: str, longitude: str) -> dict:
  """Returns weather data for a given latitude and longitude."""
  url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
  response = requests.get(url)
  print_ww(f"get_weather:tool:invoked::response={response}:")
  return response.json()

#get_weather_tool = StructuredTool.from_function(get_weather)

### Simple example of invoking Maths chain 

This next cell is to test a simple tool like Maths calculator. The response of the Bedrock models based on your constructed input. Note that we have not instrumented output to call the actual functions, but this should give you an idea on how Claude's output can be parsed and the corresponding functions can be subsequently called.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import ChatOpenAI

#from langchain.utilities import SerpAPIWrapper

modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
math_chain_llm = BedrockChat(model_id=modelId, model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

tools_list = [] #load_tools(["serpapi"], llm=react_agent_llm)

llm_math_chain = LLMMathChain(llm=math_chain_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

llm_math_chain

In [ ]:
llm_math_chain({'question': 'text 4**5'})

### Add the functions or tools as we have defined to create our `Agent`

In [11]:
tools_list = [get_lat_long,get_weather]

tools_list.append(Tool.from_function(func=llm_math_chain.run,name="Calculator",description="Useful for when you need to answer questions about math.",))

In [ ]:
for tools_s in tools_list:
    print_ww(f"Tool:name={tools_s.name}::args={tools_s.args}:: discription={tools_s.description}::")

tools_list[2].invoke({"input":(4,5)}) # test the maths calculator

### Tooling and Agents

**Use the Default prompt template**


In [ ]:
###
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


#from langchain.utilities import SerpAPIWrapper
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
react_agent_llm = BedrockChat(
    model_id=modelId,
    client=bedrock_runtime,
    #model_kwargs={"max_tokens_to_sample": 100},
    model_kwargs={"temperature": 0.1},
)

tools_list = [get_lat_long,get_weather]

tools_list.append(Tool.from_function(func=llm_math_chain.run,name="Calculator",description="Useful for when you need to answer questions about math.",))



def ask_agent(question, llm):
    react_agent = initialize_agent(
        tools_list, 
        react_agent_llm, 
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION, 
        verbose=True,
        max_iteration=4,
        return_intermediate_steps=True,
        max_execution_time=60,
        #    handle_parsing_errors=True,
        #prompt = prompt_template
    )
    print_ww(f"The prompt:template:used:was ---- > \n\n{react_agent.agent.llm_chain.prompt}\n")
    
    
    return react_agent(question)

ask_agent("can you check the weather in Marysville WA for me?", react_agent_llm)

### Tooling with Custom template

**Now Use a custom prompt template to create the Agents**

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0" 
react_agent_llm = BedrockChat(
    model_id=modelId,
    client=bedrock_runtime,
    model_kwargs={"temperature": 0.1},
)

tools_list = [get_lat_long,get_weather]

tools_list.append(Tool.from_function(func=llm_math_chain.run,name="Calculator",description="Useful for when you need to answer questions about math.",))


prompt_template_sys = """

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather", "Calculator"]
Action Input: the input to the action\nObservation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}'

"""
messages=[
    SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template=prompt_template_sys)), 
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
]

chat_prompt_template = ChatPromptTemplate.from_messages(messages)
print_ww(f"from:messages:prompt:template:{chat_prompt_template}")

chat_prompt_template = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input'], 
    messages=messages
)


react_agent = initialize_agent(
    tools_list, 
    react_agent_llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, #SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    #    handle_parsing_errors=True,
    prompt = chat_prompt_template
)

react_agent.invoke({"input":"can you check the weather in Marysville WA for me?"})

## Section 2) Leverage the create_tool_calling_agent API - 

**for bind_tools function**

This patches the bind_tools missing api to allow for tool creation. This is just a sample of how it can be done

In [15]:
#- https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/chat_models/base.py

#- bedrockChat -- https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/chat_models/bedrock.py

#- anthorpic https://github.com/langchain-ai/langchain/blob/master/libs/partners/anthropic/langchain_anthropic/chat_models.py  -- #3 has the base chat model which has the bind_tools

from __future__ import annotations



import json
from typing import (
    Any,
    Callable,
    Dict,
    List,
    Literal,
    Type,
    Union,
)

from langchain_core.pydantic_v1 import BaseModel
from langchain_core.tools import BaseTool
from langchain_core.utils.function_calling import convert_to_openai_tool
from typing_extensions import TypedDict

PYTHON_TO_JSON_TYPES = {
    "str": "string",
    "int": "integer",
    "float": "number",
    "bool": "boolean",
}

SYSTEM_PROMPT_FORMAT = """In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{formatted_tools}
</tools>"""  # noqa: E501

TOOL_FORMAT = """<tool_description>
<tool_name>{tool_name}</tool_name>
<description>{tool_description}</description>
<parameters>
{formatted_parameters}
</parameters>
</tool_description>"""

TOOL_PARAMETER_FORMAT = """<parameter>
<name>{parameter_name}</name>
<type>{parameter_type}</type>
<description>{parameter_description}</description>
</parameter>"""

def _get_type(parameter: Dict[str, Any]) -> str:
    if "type" in parameter:
        return parameter["type"]
    if "anyOf" in parameter:
        return json.dumps({"anyOf": parameter["anyOf"]})
    if "allOf" in parameter:
        return json.dumps({"allOf": parameter["allOf"]})
    return json.dumps(parameter)

def get_system_message(tools: List[AnthropicTool]) -> str:
    tools_data: List[Dict] = [
        {
            "tool_name": tool["name"],
            "tool_description": tool["description"],
            "formatted_parameters": "\n".join(
                [
                    TOOL_PARAMETER_FORMAT.format(
                        parameter_name=name,
                        parameter_type=_get_type(parameter),
                        parameter_description=parameter.get("description"),
                    )
                    for name, parameter in tool["input_schema"]["properties"].items()
                ]
            ),
        }
        for tool in tools
    ]
    tools_formatted = "\n".join(
        [
            TOOL_FORMAT.format(
                tool_name=tool["tool_name"],
                tool_description=tool["tool_description"],
                formatted_parameters=tool["formatted_parameters"],
            )
            for tool in tools_data
        ]
    )
    return SYSTEM_PROMPT_FORMAT.format(formatted_tools=tools_formatted)

class _AnthropicToolUse(TypedDict):
    type: Literal["tool_use"]
    name: str
    input: dict
    id: str
    
class AnthropicTool(TypedDict):
    name: str
    description: str
    input_schema: Dict[str, Any]

class ToolsBedrockChat(BedrockChat):

    system_prompt_with_tools: str = ""
    def convert_to_anthropic_tool(self,
        tool: Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool],
    ) -> AnthropicTool:
        # already in Anthropic tool format
        if isinstance(tool, dict) and all(
            k in tool for k in ("name", "description", "input_schema")
        ):
            return AnthropicTool(tool)  # type: ignore
        else:
            formatted = convert_to_openai_tool(tool)["function"]
            return AnthropicTool(
                name=formatted["name"],
                description=formatted["description"],
                input_schema=formatted["parameters"],
            )


    def _tools_in_params(self,params: dict) -> bool:
        return "tools" in params or (
            "extra_body" in params and params["extra_body"].get("tools")
        )
    
    def set_system_prompt_with_tools(self, xml_tools_system_prompt: str) -> None:
        """Workaround to bind. Sets the system prompt with tools"""
        self.system_prompt_with_tools = xml_tools_system_prompt

    def bind_tools(
        self,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[Union[dict, str, Literal["auto", "none"], bool]] = None,
        **kwargs: Any,
    ) -> Runnable[LanguageModelInput, BaseMessage]:
        """Bind tool-like objects to this chat model.

        Assumes model has a tool calling API.

        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be  a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require the model to call.
                Must be the name of the single provided function or
                "auto" to automatically determine which function to call
                (if any), or a dict of the form:
                {"type": "function", "function": {"name": <<tool_name>>}}.
            **kwargs: Any additional parameters to pass to the
                :class:`~langchain.runnable.Runnable` constructor.
        """
        provider = self._get_provider()

        if provider == "anthropic":
            formatted_tools = [self.convert_to_anthropic_tool(tool) for tool in tools]
            system_formatted_tools = get_system_message(formatted_tools)
            self.set_system_prompt_with_tools(system_formatted_tools)
            # llm_with_tools = self.bind(tools=system_formatted_tools, **kwargs)
        return self

    def new_func(self):
        print("pass")
    

react_agent_llm = ToolsBedrockChat(
    model_id=modelId,
    client=bedrock_runtime,
)


In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate



prompt_template_sys = """

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather", "Calculator"]
Action Input: the input to the action\nObservation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}'

"""
messages=[
    SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template=prompt_template_sys)), 
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
]

chat_prompt_template = ChatPromptTemplate.from_messages(messages)
print_ww(f"from:messages:prompt:template:{chat_prompt_template}")

chat_prompt_template = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input'], 
    messages=messages
)
print_ww(f"Crafted::prompt:template:{chat_prompt_template}")


    

#react_agent_llm.bind_tools = custom_bind_func

# Construct the Tools agent
react_agent = create_tool_calling_agent(react_agent_llm, tools_list,chat_prompt_template)
agent_executor = AgentExecutor(agent=react_agent, tools=tools_list, verbose=True)
agent_executor.invoke({"input": "can you check the weather in Marysville WA for me?"})


## Section 3 Use the Langchain-AWS classes 
These classes having all thr latest api's and working correctly

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock

modelId = "anthropic.claude-3-sonnet-20240229-v1:0" 
chat = ChatBedrock(
    model_id=modelId,
    model_kwargs={"temperature": 0.1},
    client=bedrock_runtime
)

import requests

from langchain.tools import tool
from langchain.tools import StructuredTool
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

@tool ("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params, headers=headers).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None
    
@tool ("get_weather")
def get_weather(latitude: str, longitude: str) -> dict:
  """Returns weather data for a given latitude and longitude."""
  url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
  response = requests.get(url)
  print_ww(f"get_weather:tool:invoked::response={response}:")
  return response.json()



llm_with_tools = chat.bind_tools([get_weather,get_lat_long])
print_ww(llm_with_tools)

In [ ]:
from langchain_core.messages.human import HumanMessage
messages = [
    HumanMessage(
        content="what is the weather like in San Francisco"
    )
]
ai_msg = llm_with_tools.invoke(messages)
ai_msg

In [20]:
react_agent_llm = ChatBedrock(
    model_id=modelId,
    client=bedrock_runtime,
    #model_kwargs={"max_tokens_to_sample": 100},
    #model_kwargs={"temperature": 0.1},
)

#### Create your own template with langchain-aws

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate



prompt_template_sys = """

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather", "Calculator"]
Action Input: the input to the action\nObservation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}'

"""
messages=[
    SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template=prompt_template_sys)), 
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))
]

chat_prompt_template = ChatPromptTemplate.from_messages(messages)
print_ww(f"from:messages:prompt:template:{chat_prompt_template}")

chat_prompt_template = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input'], 
    messages=messages
)
print_ww(f"Crafted::prompt:template:{chat_prompt_template}")


    

#react_agent_llm.bind_tools = custom_bind_func

# Construct the Tools agent
react_agent = create_tool_calling_agent(react_agent_llm, tools_list,chat_prompt_template)
agent_executor = AgentExecutor(agent=react_agent, tools=tools_list, verbose=True)
agent_executor.invoke({"input": "can you check the weather in Marysville WA for me?"})

#### Creating the chain of Agents + tools manually

If you want to create the Chat

In [22]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool

from langchain.agents.format_scratchpad.tools import (
    format_to_tool_messages,
)
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

llm_with_tools = react_agent_llm.bind_tools(tools_list)

conversational_agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
        )
        | chat_prompt_template
        | llm_with_tools
        | ToolsAgentOutputParser()
)

In [ ]:
output_t = conversational_agent.invoke({"input": "What is Amazon SageMaker Clarify?", "intermediate_steps": []})

print_ww(output_t)

## Next steps

In this notebook we showed some basic examples of leveraging tools and agents when invoking Amazon Bedrock models using the AWS Python SDK. You're now ready to explore the other labs to dive deeper on different use-cases and patterns.